In [1]:
import open3d as o3d
import numpy as np

from skspatial.objects import Plane, Points
from skspatial.plotting import plot_3d
import matplotlib.pyplot as plt
import matplotlib
import time
from PIL import Image

import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcds: list[o3d.geometry.PointCloud] = []
# plys_dir = 'very_flat_plys/'
plys_dir = 'road_long_towards_plys/'

for fn in os.listdir(plys_dir):
    pcds.append(o3d.io.read_point_cloud(plys_dir + fn))

In [3]:
inliners = []
inliners_colors = []
outliners = []
outliners_err = []

for i, pcd in enumerate(pcds):
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=10, std_ratio=1.0)

    plane_model, inliers = pcd.segment_plane(distance_threshold=0.1,
                                             ransac_n=3,
                                             num_iterations=1000)

    plane_model_err, inliers_err = pcd.segment_plane(distance_threshold=0.15,
                                             ransac_n=3,
                                             num_iterations=1000)

    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    outlier_cloud_err = pcd.select_by_index(inliers_err, invert=True)
    outlier_cloud.paint_uniform_color([0.8, 0.5, 0])
    outlier_cloud_err.paint_uniform_color([1, 0, 0])

    inliners.append(np.asarray(inlier_cloud.points))
    inliners_colors.append(np.asarray(inlier_cloud.colors))
    outliners.append(np.asarray(outlier_cloud.points))
    outliners_err.append(np.asarray(outlier_cloud_err.points))

    if i % 100 == 0:
        print(f'Finished {i+1}/{len(pcds)}')

inliners = np.concatenate(inliners)
inliners_colors = np.concatenate(inliners_colors)
outliners = np.concatenate(outliners)
outliners_err = np.concatenate(outliners_err)

Finished 1/811
Finished 2/811
Finished 3/811
Finished 4/811
Finished 5/811
Finished 6/811
Finished 7/811
Finished 8/811
Finished 9/811
Finished 10/811
Finished 11/811
Finished 12/811
Finished 13/811
Finished 14/811
Finished 15/811
Finished 16/811
Finished 17/811
Finished 18/811
Finished 19/811
Finished 20/811
Finished 21/811
Finished 22/811
Finished 23/811
Finished 24/811
Finished 25/811
Finished 26/811
Finished 27/811
Finished 28/811
Finished 29/811
Finished 30/811
Finished 31/811
Finished 32/811
Finished 33/811
Finished 34/811
Finished 35/811
Finished 36/811
Finished 37/811
Finished 38/811
Finished 39/811
Finished 40/811
Finished 41/811
Finished 42/811
Finished 43/811
Finished 44/811
Finished 45/811
Finished 46/811
Finished 47/811
Finished 48/811
Finished 49/811
Finished 50/811
Finished 51/811
Finished 52/811
Finished 53/811
Finished 54/811
Finished 55/811
Finished 56/811
Finished 57/811
Finished 58/811
Finished 59/811
Finished 60/811
Finished 61/811
Finished 62/811
Finished 63/811
F

In [4]:
inliners_pcd = o3d.geometry.PointCloud()
inliners_pcd.points = o3d.utility.Vector3dVector(inliners)
inliners_pcd.colors = o3d.utility.Vector3dVector(inliners_colors)

outliners_pcd = o3d.geometry.PointCloud()
outliners_pcd.points = o3d.utility.Vector3dVector(outliners)
outliners_pcd.paint_uniform_color([0.8, 0.5, 0.0])

outliners_err_pcd = o3d.geometry.PointCloud()
outliners_err_pcd.points = o3d.utility.Vector3dVector(outliners_err)
outliners_err_pcd.paint_uniform_color([1.0, 0.0, 0.0])

PointCloud with 84928 points.

In [6]:
o3d.visualization.draw_geometries([inliners_pcd, outliners_pcd, outliners_err_pcd])

In [23]:
def flatten(pcd: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
    pts = np.asarray(pcd.points).copy()
    colors = np.asarray(pcd.colors).copy()
    for pt in pts:
        pt[2] = 0.0

    new_pcd = o3d.geometry.PointCloud()
    new_pcd.points = o3d.utility.Vector3dVector(pts)
    new_pcd.colors = o3d.utility.Vector3dVector(colors)

    return new_pcd

flat_inliners = flatten(inliners_pcd)
flat_outliners = flatten(outliners_pcd)
flat_outliners_err = flatten(outliners_err_pcd)

In [24]:
# flat_inliners_filt, _ = flat_inliners.remove_radius_outlier(50, 0.5)
flat_outliners_filt, _ = flat_outliners.remove_radius_outlier(50, 0.5)
flat_outliners_err_filt, _ = flat_outliners_err.remove_radius_outlier(50, 0.5)

In [9]:
o3d.visualization.draw_geometries([flat_inliners, flat_outliners_filt, flat_outliners_err_filt])

In [ ]:
### Taking picture, not working really well

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(flat_inliners)
vis.add_geometry(flat_outliners_filt)
vis.add_geometry(flat_outliners_err_filt)
vis.update_geometry(flat_inliners)
vis.update_geometry(flat_outliners_filt)
vis.update_geometry(flat_outliners_err_filt)
vis.poll_events()
vis.update_renderer()
vis.capture_screen_image('image.png')
vis.destroy_window()

In [21]:
# R = flat_inliners.get_rotation_matrix_from_xyz((0.0, 0.0, -np.pi / 4))
# flat_inliners.rotate(R)
# flat_outliners_filt.rotate(R)
# flat_outliners_err_filt.rotate(R)

PointCloud with 77589 points.

In [26]:
pcd_combined = flat_inliners +  flat_outliners_filt + flat_outliners_err_filt
assert isinstance(pcd_combined, o3d.geometry.PointCloud)
if plys_dir == 'road_long_towards_plys/':
    # rotate to make it more horizontal for image
    R = flat_inliners.get_rotation_matrix_from_xyz((0.0, 0.0, -np.pi / 4))
    pcd_combined.rotate(R)
o3d.visualization.draw_geometries([pcd_combined])

In [27]:
# inliners_points = np.asarray(flat_inliners.points).copy()
# inliners_colors = np.asarray(flat_inliners.colors).copy()
# outliners_points = np.asarray(flat_outliners_filt.points).copy()
# outliners_colors = np.asarray(flat_outliners_filt.colors).copy()
# outliners_err_points = np.asarray(flat_outliners_err_filt.points).copy()
# outliners_err_colors = np.asarray(flat_outliners_err_filt.colors).copy()

# all_points = np.concatenate([inliners_points, outliners_points, outliners_err_points])
# all_colors = np.concatenate([inliners_colors, outliners_colors, outliners_err_colors])
all_points = np.asarray(pcd_combined.points).copy()
all_colors = np.asarray(pcd_combined.colors).copy()

In [28]:
max_dim = all_points.max(axis=0)
min_dim = all_points.min(axis=0)
span = (max_dim - min_dim)[:2]

shifted_points = all_points - min_dim

print(span)

[733.52038745  94.92623692]


In [29]:
img_scale = 10
img_size = tuple((img_scale * span).astype('int') + 1)
print(img_size)
img = Image.new('RGB', img_size, color=(100, 100, 100))
pixels = img.load()

In [30]:
for i, (point, color) in enumerate(zip(shifted_points, all_colors)):
    point = tuple((img_scale * point[:2]).astype('int'))
    color = tuple((256 * color).astype('int'))
    pixels[point] = color

    if i % 100000 == 0:
        print(f'{i}/{len(shifted_points)}')
print('Finished')

0/13025803
100000/13025803
200000/13025803
300000/13025803
400000/13025803
500000/13025803
600000/13025803
700000/13025803
800000/13025803
900000/13025803
1000000/13025803
1100000/13025803
1200000/13025803
1300000/13025803
1400000/13025803
1500000/13025803
1600000/13025803
1700000/13025803
1800000/13025803
1900000/13025803
2000000/13025803
2100000/13025803
2200000/13025803
2300000/13025803
2400000/13025803
2500000/13025803
2600000/13025803
2700000/13025803
2800000/13025803
2900000/13025803
3000000/13025803
3100000/13025803
3200000/13025803
3300000/13025803
3400000/13025803
3500000/13025803
3600000/13025803
3700000/13025803
3800000/13025803
3900000/13025803
4000000/13025803
4100000/13025803
4200000/13025803
4300000/13025803
4400000/13025803
4500000/13025803
4600000/13025803
4700000/13025803
4800000/13025803
4900000/13025803
5000000/13025803
5100000/13025803
5200000/13025803
5300000/13025803
5400000/13025803
5500000/13025803
5600000/13025803
5700000/13025803
5800000/13025803
5900000/1302

In [31]:
img.show()

In [32]:
img.save('map.png')

In [38]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(flat_outliners_err_filt.cluster_dbscan(eps=0.1, min_points=50, print_progress=True))
print(labels.max())

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.====================>] 97%
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 238
Precompute neighbors.[========================================] 100%
237stering[=>                                      ] 2%


In [39]:
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
flat_outliners_err_filt.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([flat_outliners_err_filt])